In [ ]:
# auteurs:
#     KODZO Michel
#     SAKPA Freeman 
#     projets scolaires

In [ ]:
#importation des bibliotheque

import numpy as np
import pandas as pd
import sklearn 
from sklearn.model_selection import train_test_split 

import matplotlib.pyplot as plt
print(pd.__version__)
print(np.__version__)
print(sklearn.__version__)




In [ ]:
#affichage des données
housing = pd.read_csv('./housing.csv')
housing.head()

In [ ]:
housing.info()

In [ ]:
# categorisation des distances
housing["ocean_proximity"].value_counts()

In [ ]:
#resumé des donnés
housing.describe()

In [ ]:
#histogrammes des donnés
housing.hist(bins=50, figsize=(20,15))
plt.show()


In [ ]:
#on reserve 20% des ressources pour le test
from sklearn.model_selection import train_test_split
train_set, test_set = train_test_split(housing, test_size=0.2, random_state=42)


print(len(train_set), "données entrainement +", len(test_set), "données test")

In [ ]:
housing["income_cat"] = np.ceil(housing["median_income"] / 1.5)
housing["income_cat"].where(housing["income_cat"] < 5, 5.0, inplace=True)

from sklearn.model_selection import StratifiedShuffleSplit
split = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)

for train_index, test_index in split.split(housing, housing["income_cat"]):
    strat_train_set = housing.loc[train_index]
    strat_test_set = housing.loc[test_index]
strat_train_set.head()

In [ ]:
housing["income_cat"].value_counts() /len(housing)
# drop the income_cat column
for set in (strat_train_set, strat_test_set):
    set.drop(["income_cat"], axis=1, inplace=True)
strat_train_set.head()

In [ ]:


housing = strat_train_set.copy()
housing.plot(kind="scatter", x="longitude", y="latitude", alpha=0.4, s=housing["population"]/100, 
             label="population", c="median_house_value", cmap=plt.get_cmap("jet"), colorbar=True,
            )
plt.legend()
housing.head()

In [ ]:
#matrice de dorrelation
corr_matrix = housing.corr()
corr_matrix["median_house_value"].sort_values(ascending=False)

In [ ]:
#matrice de dispersion
from pandas.plotting import scatter_matrix
attributes = ["median_house_value", "median_income", "total_rooms", "housing_median_age"]
scatter_matrix(housing[attributes], figsize=(12,8))

In [ ]:
housing.plot(kind="scatter", x="median_income", y="median_house_value", alpha=0.1)

In [ ]:
housing["rooms_per_household"] = housing["total_rooms"] / housing["households"]
housing["bedrooms_per_room"] = housing["total_bedrooms"] / housing["total_rooms"]
housing["population_per_household"] = housing["population"] / housing["households"]
#corr_matrix = housing.corr()
#corr_matrix["median_house_value"].sort_values(ascending=False)
housing.head()

In [ ]:
# creer X et y 
housing = strat_train_set.drop("median_house_value", axis=1)
housing_labels = strat_train_set["median_house_value"].copy()
housing.head()

In [ ]:
#fill median value for missing total_bedrooms
median = housing["total_bedrooms"].median()
housing["total_bedrooms"].fillna(median)
housing.info()
print(median)


In [ ]:
housing_cat = housing["ocean_proximity"]
from sklearn.preprocessing import LabelBinarizer
encoder = LabelBinarizer()
housing_cat_1hot = encoder.fit_transform(housing_cat)
housing_cat_1hot

In [28]:
from sklearn.base import BaseEstimator, TransformerMixin

rooms_ix, bedroom_ix, population_ix, household_ix = 3,4,5,6

class CombinedAttributesAdder(BaseEstimator, TransformerMixin):
    def __init__(self, add_bedrooms_per_room = True):
        self.add_bedrooms_per_room = add_bedrooms_per_room
    def fit(self, X, y=None):
        return self
    def transform(self, X, y=None):
        rooms_per_household = X[:, rooms_ix] / X[:, household_ix]
        population_per_household = X[:, population_ix] / X[:, household_ix]
        if self.add_bedrooms_per_room:
            bedrooms_per_room = X[:, bedroom_ix] / X[:, rooms_ix]
            return np.c_[X, rooms_per_household, population_per_household, bedrooms_per_room]
        else:
            return np.c_[X, rooms_per_household, population_per_household]

attr_adder = CombinedAttributesAdder(add_bedrooms_per_room=False)
housing_extra_attribs = attr_adder.transform(housing.values)
housing_extra_attribs[0]

array([-121.46, 38.52, 29.0, 3873.0, 797.0, 2237.0, 706.0, 2.1736,
       'INLAND', 5.485835694050992, 3.168555240793201], dtype=object)

In [ ]:
housing.head()

In [34]:
housing_num = housing.drop("ocean_proximity", axis=1)
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.pipeline import FeatureUnion

class DataFrameSelector(BaseEstimator, TransformerMixin):
    def __init__(self, attribute_names):
        self.attribute_names = attribute_names
    def fit(self, X, y=None):
        return self
    def transform(self, X):
        return X[self.attribute_names].values


class CustomBinarizer(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None,**fit_params):
        return self
    def transform(self, X):
        return LabelBinarizer().fit(X).transform(X)

num_attribs = list(housing_num)
cat_attribs = ["ocean_proximity"]

num_pipeline = Pipeline([
    ('selector', DataFrameSelector(num_attribs)),
    ('imputer', SimpleImputer(strategy="median")),
    ('attributes_adder', CombinedAttributesAdder()),
    ('std_scaler', StandardScaler()),
])

cat_pipeline = Pipeline ([
    ('selector', DataFrameSelector(cat_attribs)),
    ('custom_binarizer', CustomBinarizer() ),
#    ('label_binarizer', LabelBinarizer()),
])

full_pipeline = FeatureUnion(transformer_list=[
    ("num_pipeline", num_pipeline),
    ("cat_pipeline", cat_pipeline),
])

# run the pipeline
housing_prepared = full_pipeline.fit_transform(housing)
housing_prepared[0,:]

array([-0.94135046,  1.34743822,  0.02756357,  0.58477745,  0.64037127,
        0.73260236,  0.55628602, -0.8936472 ,  0.01739526,  0.00622264,
       -0.12112176,  0.        ,  1.        ,  0.        ,  0.        ,
        0.        ])

In [35]:

from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
lin_reg = LinearRegression()
lin_reg.fit(housing_prepared, housing_labels)


housing_predictions = lin_reg.predict(housing_prepared)
lin_mse = mean_squared_error(housing_labels, housing_predictions)
lin_rmse = np.sqrt(lin_mse)
lin_rmse

68627.87390018745

In [36]:
from sklearn.tree import DecisionTreeRegressor
tree_reg = DecisionTreeRegressor()
tree_reg.fit(housing_prepared, housing_labels)
housing_predictions = tree_reg.predict(housing_prepared)
tree_mse = mean_squared_error(housing_labels, housing_predictions)
tree_rmse = np.sqrt(tree_mse)
tree_rmse


0.0

In [37]:
from sklearn.model_selection import cross_val_score
scores = cross_val_score(tree_reg, housing_prepared, housing_labels, scoring="neg_mean_squared_error",
                        cv=10)
tree_rmse_scores = np.sqrt(-scores)
def display_scores(scores):
    print("scores:", scores)
    print("Mean:", scores.mean())
    print("Standard Deviation:", scores.std())
    
display_scores(tree_rmse_scores)

scores: [71386.6925826  69882.02945132 68772.19768986 69825.33168067
 69913.60650546 78248.80891723 71608.86055875 73189.66098203
 67252.87381503 70865.53746553]
Mean: 71094.55996484568
Standard Deviation: 2839.0626091040563


In [38]:
# score de la regression lineaire
lin_scores = cross_val_score(lin_reg, housing_prepared, housing_labels, scoring="neg_mean_squared_error",
                        cv=10)

lin_rmse_scores = np.sqrt(-lin_scores)
display_scores(lin_rmse_scores)

scores: [71762.76364394 64114.99166359 67771.17124356 68635.19072082
 66846.14089488 72528.03725385 73997.08050233 68802.33629334
 66443.28836884 70139.79923956]
Mean: 69104.07998247063
Standard Deviation: 2880.3282098180634


In [39]:
# Random Forest
from sklearn.ensemble import RandomForestRegressor
forest_reg = RandomForestRegressor()
forest_reg.fit(housing_prepared, housing_labels)
housing_predictions = forest_reg.predict(housing_prepared)
forest_mse = mean_squared_error(housing_labels, housing_predictions)
forest_rmse = np.sqrt(forest_mse)
forest_rmse

forest_scores = cross_val_score(forest_reg, housing_prepared, housing_labels, 
                                scoring="neg_mean_squared_error",cv=10)
forest_rmse_scores = np.sqrt(-forest_scores)
display_scores(forest_rmse_scores)

scores: [51716.52876368 49114.80898285 46722.07951914 52306.24560286
 47558.2227504  51704.79227968 52314.04795392 49769.14333471
 48748.34086446 53804.64711163]
Mean: 50375.88571633351
Standard Deviation: 2206.7931642458807


In [40]:
from sklearn.model_selection import GridSearchCV
param_grid = [
    {'n_estimators': [3,10,30], 'max_features': [2,4,6,8]},
    {'bootstrap': [False], 'n_estimators': [3,10], 'max_features': [2,3,4]}
]
forest_reg = RandomForestRegressor()
grid_search = GridSearchCV(forest_reg, param_grid, cv=5, scoring='neg_mean_squared_error')
grid_search.fit(housing_prepared, housing_labels)
grid_search.best_params_
grid_search.best_estimator_

RandomForestRegressor(max_features=6, n_estimators=30)

In [41]:
grid_search.best_params_

{'max_features': 6, 'n_estimators': 30}

In [42]:
grid_search.best_params_
grid_search.best_estimator_
cvres = grid_search.cv_results_
for mean_score, params in zip(cvres["mean_test_score"], cvres["params"]):
    print(np.sqrt(-mean_score), params)

62964.647810043294 {'max_features': 2, 'n_estimators': 3}
55491.8758005023 {'max_features': 2, 'n_estimators': 10}
52928.99728840813 {'max_features': 2, 'n_estimators': 30}
59737.21927208477 {'max_features': 4, 'n_estimators': 3}
52826.669305697484 {'max_features': 4, 'n_estimators': 10}
50329.547186184216 {'max_features': 4, 'n_estimators': 30}
58747.28429335071 {'max_features': 6, 'n_estimators': 3}
52352.85426099821 {'max_features': 6, 'n_estimators': 10}
49969.35789409789 {'max_features': 6, 'n_estimators': 30}
58700.3352760964 {'max_features': 8, 'n_estimators': 3}
52042.22671189716 {'max_features': 8, 'n_estimators': 10}
50145.0798463569 {'max_features': 8, 'n_estimators': 30}
62254.618269118335 {'bootstrap': False, 'max_features': 2, 'n_estimators': 3}
54109.55977528157 {'bootstrap': False, 'max_features': 2, 'n_estimators': 10}
60668.96347400298 {'bootstrap': False, 'max_features': 3, 'n_estimators': 3}
52743.53742641 {'bootstrap': False, 'max_features': 3, 'n_estimators': 10}


In [43]:
#importance de chaque attributs
feature_importance = grid_search.best_estimator_.feature_importances_
feature_importance

array([7.56624611e-02, 6.54089649e-02, 4.50608016e-02, 1.84670936e-02,
       1.64423712e-02, 1.82861490e-02, 1.65610165e-02, 3.33513633e-01,
       6.52697018e-02, 1.07141353e-01, 7.36395856e-02, 1.24591423e-02,
       1.41944578e-01, 4.14489333e-05, 4.04193155e-03, 6.05976823e-03])

In [44]:
extra_attribs = ["rooms_per_hhold", "pop_per_hhold", "bedrooms_per_room"]
cat_one_hot_attribs = list(encoder.classes_)
cat_one_hot_attribs
attributes = num_attribs + extra_attribs + cat_one_hot_attribs
sorted (zip(feature_importance, attributes), reverse=True)

[(0.33351363258240224, 'median_income'),
 (0.14194457765226923, 'INLAND'),
 (0.10714135346262091, 'pop_per_hhold'),
 (0.0756624610782566, 'longitude'),
 (0.07363958556221671, 'bedrooms_per_room'),
 (0.06540896487650785, 'latitude'),
 (0.0652697018284685, 'rooms_per_hhold'),
 (0.0450608015754875, 'housing_median_age'),
 (0.018467093611770687, 'total_rooms'),
 (0.018286149020844068, 'population'),
 (0.016561016543973534, 'households'),
 (0.01644237123395746, 'total_bedrooms'),
 (0.012459142256604836, '<1H OCEAN'),
 (0.00605976822858613, 'NEAR OCEAN'),
 (0.004041931552738025, 'NEAR BAY'),
 (4.144893329567038e-05, 'ISLAND')]

In [45]:
#  evaluation du model
final_model = grid_search.best_estimator_
X_test = strat_test_set.drop("median_house_value", axis = 1)
y_test = strat_test_set["median_house_value"].copy()

X_test_prepared = full_pipeline.transform(X_test)
final_predictions = final_model.predict(X_test_prepared)
final_mse = mean_squared_error(y_test, final_predictions)
final_rmse = np.sqrt(final_mse)
final_rmse

48256.316630275076